In [1]:
import torch
import transformers
import diffusers
import pandas as pd
from PIL import Image
import torch

In [2]:
import os
import torch
from tqdm.notebook import tqdm
import numpy as np
import transformers
from diffusers import StableDiffusionPipeline, DDPMScheduler
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import CLIPTokenizer, CLIPTextModel, set_seed
from pathlib import Path
import logging
from diffusers.models import UNet2DConditionModel, AutoencoderKL
from peft import LoraConfig

In [3]:
# pipe = diffusers.DiffusionPipeline.from_pretrained('data/model_bins/stable-diffusion-xl-base-1.0')

In [4]:
# pipe.to("cuda")


In [5]:
parquet_dataset = pd.read_parquet('data/laion-pop/laion_pop.parquet')
parquet_dataset

,index,key,sha256,url,llava_caption,nsfw_prediction,alt_txt,alt_txt_similarity,width,height,original_width,original_height,exif
0,0,000623232,d46d20ff9fa7212b3671b49ae73371672711e5265fff29...,https://img1.etsystatic.com/000/0/5370909/il_f...,a black and white drawing of a man climbing a ...,0.000050,Art Deco Man Cave : Rockwell kent art deco man...,0.592961,1000.0,1013.0,1000.0,1013.0,{}
1,1,000616883,dde6e39a297da2d7e8923bdf569c0e4704b29a69148b9d...,https://www.aktay.net/wp-content/uploads/2014/...,a magazine or brochure that is open to a page ...,0.000372,pergo broşür tasarımı 12,0.644895,1600.0,1143.0,1600.0,1143.0,{}
2,2,000620777,7196431abc8b8fabfa34f50be738ba1c0a4c72ccbc70ef...,https://cdn5.vectorstock.com/i/1000x1000/03/14...,a gold and black building logo. The logo featu...,0.000272,Round building company gold logo,0.537708,956.0,1080.0,956.0,1080.0,{}
3,3,000613640,62129823dacee6358a619a9ca25f0f9c069efa480a0dab...,https://cdn.lifestyleasia.com/wp-content/uploa...,a cozy living room with a Christmas tree and a...,0.000678,"10 easy, no humbug tips for that perfect Chris...",0.722248,5258.0,3505.0,5258.0,3505.0,{}
4,4,000621773,3620f54366401bd7739750d1e85b64ca41020497117ea7...,https://image.invaluable.com/housePhotos/galle...,a bird statue that is perched on top of a brow...,0.000138,Chinese Export Sterling Enameled Hawk on Base,0.555684,1000.0,1237.0,1000.0,1237.0,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...
591877,646166,004161450,37050df597ddbe1ad6f7e68ea49850af2b952dc34faaac...,https://cdn.shopify.com/s/files/1/0106/5235/25...,"a woman wearing a pink dress, standing outside...",0.986169,Bohemian printed beach skirt female hanging ne...,0.825665,800.0,1000.0,800.0,1000.0,"{""Image HostComputer"": ""imagery4""}"
591878,646167,004161435,f271089838b7a6695d50370a920893f088ac3e86b85018...,https://profile-images.xing.com/images/c645457...,a man wearing glasses and a green shirt. He is...,0.000191,Felix roth gesch ftsf hrer kommunikationsdesig...,0.612309,1024.0,1024.0,1024.0,1024.0,{}
591879,646168,004161361,2fafded547cd8cf195be31836eba24c65a62fbc33021bd...,https://images.unsplash.com/photo-151232829114...,a man standing on a beach at sunset. He is wea...,0.000034,photography of man standing near water body,0.505804,6720.0,4480.0,6720.0,4480.0,{}
591880,646169,004161484,3d730cbb0cea2da7c63e1bab5a4b8c883561d78b83e1a3...,https://www.gannett-cdn.com/presto/2019/05/10/...,"a large, colorful carousel in a carnival setti...",0.000040,Mall patrons walk past the 25-foot tall carous...,0.544864,2595.0,1921.0,2595.0,1921.0,{}


In [6]:
def get_image_and_caption(parquet, key):
    caption = parquet[parquet['key'] == key]['llava_caption']
    caption = caption[caption.index[0]]
    image = Image.open(f'data/images/{key}.jpg')
    return image, caption

In [7]:
# img, cap = get_image_and_caption(parquet_dataset, '000620537')

In [8]:
# from transformers import CLIPTokenizer, CLIPTextModel, set_seed

# model_id = "data/model_bins/stable-diffusion-xl-base-1.0/tokenizer"  # Replace with your model ID
# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = CLIPTokenizer.from_pretrained(model_id)

In [9]:
import glob

path_to_pics = 'data/images/'
all_pics_paths = glob.glob(path_to_pics + '*')
len(all_pics_paths)

4464

In [10]:
from torch.utils.data import Dataset 
from torchvision import transforms
from torchvision.transforms.functional import crop

class LaionHQ(Dataset):
    def __init__(self, dataframe, path_to_pics):
        self.dataframe = dataframe
        self.path_to_pics = path_to_pics
        all_pics_paths = glob.glob(path_to_pics + '*')
        self.unique_imgs = [i.split('/')[-1].split('.')[-2] for i in all_pics_paths]
        self.dataframe.index = self.dataframe.key
        
        self.train_resize = transforms.Resize(1024, interpolation=transforms.InterpolationMode.BILINEAR)
        self.train_crop = transforms.CenterCrop(1024) if False else transforms.RandomCrop(1024)
        # self.train_flip = transforms.RandomHorizontalFlip(p=1.0)
        
        self.train_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

    def __len__(self):
        return len(self.unique_imgs)
    
    def __getitem__(self, idx):
        key = self.unique_imgs[idx]
        path_to_img = os.path.join(self.path_to_pics, key + '.jpg')
        image = Image.open(path_to_img).convert('RGB')
        original_size = (image.height, image.width)
        
        image = self.train_resize(image)
        
        y1, x1, h, w = self.train_crop.get_params(image, (1024, 1024))
        image = crop(image, y1, x1, h, w)
            
        image = self.train_crop(image)
        # image = self.train_flip(image)
        pixel_values = self.train_transforms(image)
        crop_top_left = (y1, x1)

        return  pixel_values, original_size, crop_top_left, self.dataframe.loc[key].llava_caption

In [11]:
# from torchvision import transforms

# train_resize = transforms.Resize(args.resolution, interpolation=transforms.InterpolationMode.BILINEAR)
# train_crop = transforms.CenterCrop(args.resolution) if args.center_crop else transforms.RandomCrop(args.resolution)
# train_flip = transforms.RandomHorizontalFlip(p=1.0)
# train_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])


In [12]:
dataset = LaionHQ(parquet_dataset, 'data/images/')

In [13]:
next(iter(dataset))

(tensor([[[-0.9059, -0.9137, -0.9137,  ..., -0.7098, -0.7098, -0.7098],
          [-0.8980, -0.9059, -0.9059,  ..., -0.7098, -0.7098, -0.7020],
          [-0.8824, -0.8902, -0.8980,  ..., -0.7098, -0.7098, -0.7020],
          ...,
          [-0.9137, -0.9137, -0.9137,  ..., -0.7020, -0.7020, -0.6941],
          [-0.9137, -0.9137, -0.9137,  ..., -0.7098, -0.7098, -0.7098],
          [-0.9137, -0.9137, -0.9137,  ..., -0.7098, -0.7098, -0.7098]],
 
         [[-0.9059, -0.9137, -0.9059,  ..., -0.6784, -0.6784, -0.6784],
          [-0.8980, -0.9059, -0.8980,  ..., -0.6784, -0.6784, -0.6706],
          [-0.8824, -0.8902, -0.8980,  ..., -0.6784, -0.6784, -0.6706],
          ...,
          [-0.9137, -0.9137, -0.9137,  ..., -0.7412, -0.7412, -0.7333],
          [-0.9137, -0.9137, -0.9137,  ..., -0.7490, -0.7490, -0.7490],
          [-0.9137, -0.9137, -0.9137,  ..., -0.7490, -0.7490, -0.7490]],
 
         [[-0.9059, -0.9137, -0.9137,  ..., -0.6863, -0.6863, -0.6863],
          [-0.8980, -0.9059,

In [14]:

logging.basicConfig(level=logging.INFO)
set_seed(42)  # Replace with your seed


model_path = "data/model_bins/stable-diffusion-xl-base-1.0"  # Replace with your model ID
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_one = CLIPTokenizer.from_pretrained(os.path.join(model_path, 'tokenizer'))
tokenizer_two = CLIPTokenizer.from_pretrained(os.path.join(model_path, 'tokenizer_2'))

text_encoder_one = CLIPTextModel.from_pretrained(os.path.join(model_path, 'text_encoder'))
text_encoder_two = CLIPTextModel.from_pretrained(os.path.join(model_path, 'text_encoder_2'))

vae = AutoencoderKL.from_pretrained(os.path.join(model_path, 'vae')).to(device)
unet = UNet2DConditionModel.from_pretrained(os.path.join(model_path, 'unet')).to(device)
noise_scheduler = DDPMScheduler.from_pretrained(model_path, subfolder="scheduler")


In [15]:
for param in unet.parameters():
    param.requires_grad = False

In [16]:
total_params = 0
for param in unet.parameters():
    if param.requires_grad:
        total_params += param.numel()
total_params

0

In [17]:
unet_lora_config = LoraConfig(
    r=8, init_lora_weights="gaussian", target_modules=["to_k", "to_q", "to_v", "to_out.0"]
)

unet.add_adapter(unet_lora_config)

# The text encoder comes from 🤗 transformers, we will also attach adapters to it.

    # ensure that dtype is float32, even if rest of the model that isn't trained is loaded in fp16
# text_lora_config = LoraConfig(
#     r=8, init_lora_weights="gaussian", target_modules=["q_proj", "k_proj", "v_proj", "out_proj"]
# )
# text_encoder.add_adapter(text_lora_config)


In [18]:
# text_encoder.add_adapter(text_lora_config)

In [19]:
# total_params = 0
# for param in text_encoder.parameters():
#     if param.requires_grad:
#         total_params += param.numel()
# total_params

In [20]:
total_params = 0
for param in unet.parameters():
    if param.requires_grad:
        total_params += param.numel()
total_params

11612160

In [56]:
text_encoder_one = text_encoder_one.cuda()
text_encoder_two = text_encoder_two.cuda()

In [64]:
def tokens_to_device(token_dict, device):
    for key in token_dict:
        token_dict[key] = token_dict[key].to(device)
    return token_dict

def tokenize_prompt(tokenizer, prompt):
    text_inputs = tokenizer(
        prompt,
        padding="max_length",
        max_length=tokenizer.model_max_length,
        truncation=True,
        return_tensors="pt",
    )
    text_input_ids = text_inputs.input_ids
    return text_input_ids

def encode_prompt(text_encoders, tokenizers, prompt, text_input_ids_list=None):
    prompt_embeds_list = []

    for i, text_encoder in enumerate(text_encoders):
        if tokenizers is not None:
            tokenizer = tokenizers[i]
            text_input_ids = tokenize_prompt(tokenizer, prompt)
        else:
            assert text_input_ids_list is not None
            text_input_ids = text_input_ids_list[i]

        prompt_embeds = text_encoder(
            text_input_ids.to(text_encoder.device),
            output_hidden_states=True,
        )

        # We are only ALWAYS interested in the pooled output of the final text encoder
        pooled_prompt_embeds = prompt_embeds[0]
        prompt_embeds = prompt_embeds.hidden_states[-2]
        bs_embed, seq_len, _ = prompt_embeds.shape
        prompt_embeds = prompt_embeds.view(bs_embed, seq_len, -1)
        prompt_embeds_list.append(prompt_embeds)

    prompt_embeds = torch.concat(prompt_embeds_list, dim=-1)
    pooled_prompt_embeds = pooled_prompt_embeds.view(bs_embed, -1)
    return prompt_embeds, pooled_prompt_embeds

def compute_vae_encodings(images, vae):
    pixel_values = torch.stack(list(images))
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    pixel_values = pixel_values.to(vae.device, dtype=vae.dtype)

    with torch.no_grad():
        model_input = vae.encode(pixel_values).latent_dist.sample()
    model_input = model_input * vae.config.scaling_factor
    return model_input


In [65]:
def custom_collate_fn(batch):
    """
    Return batch.keys() == model_input, original_sizes, crop_top_lefts, prompt_embeds, pooled_prompt_embeds
    """
    output = {}
    images, original_sizes, crop_top_lefts, prompts = zip(*batch)
    
    images = torch.stack(images)
    # model_input = compute_vae_encodings(images, vae)
    # output.update(model_input)
    output.update(
        {
            'img_tensors': images,
            'prompts': prompts
        }
    )
    output.update({
        'original_sizes' : original_sizes,
        'crop_top_lefts' : crop_top_lefts,
    })
    # output.update(encode_prompt(prompts, text_encoders, tokenizers))
    
    return output

In [66]:
text_encoders = [text_encoder_one, text_encoder_two]
tokenizers = [tokenizer_one, tokenizer_two]

In [85]:
train_dataloader = DataLoader(dataset, collate_fn = custom_collate_fn, num_workers = 8, batch_size=1, shuffle=True)


In [86]:
next(iter(train_dataloader))

{'img_tensors': tensor([[[[ 0.0588,  0.1216,  0.1608,  ...,  0.3255,  0.4118,  0.4745],
           [ 0.0745,  0.1137,  0.1529,  ...,  0.3255,  0.3804,  0.4431],
           [ 0.0431,  0.0902,  0.1451,  ...,  0.3255,  0.3569,  0.3961],
           ...,
           [-0.7804, -0.7961, -0.8196,  ...,  0.4275,  0.4353,  0.4431],
           [-0.7882, -0.7961, -0.8275,  ...,  0.4039,  0.4275,  0.4118],
           [-0.7804, -0.7961, -0.8275,  ...,  0.4118,  0.4118,  0.4039]],
 
          [[ 0.1137,  0.1765,  0.2235,  ...,  0.4118,  0.4196,  0.4275],
           [ 0.1294,  0.1686,  0.2157,  ...,  0.4118,  0.4039,  0.4196],
           [ 0.1059,  0.1529,  0.2235,  ...,  0.4118,  0.4039,  0.4118],
           ...,
           [-0.8902, -0.8980, -0.9216,  ...,  0.3020,  0.3176,  0.3176],
           [-0.8745, -0.8824, -0.9137,  ...,  0.2941,  0.3176,  0.2941],
           [-0.8588, -0.8745, -0.9137,  ...,  0.3098,  0.3020,  0.2863]],
 
          [[ 0.0510,  0.1137,  0.1529,  ...,  0.2863,  0.2627,  0.2549]

In [87]:
from diffusers.optimization import get_scheduler
import math
lr_warmup_steps = 500
epochs = 1
num_update_steps_per_epoch = math.ceil(len(train_dataloader))

max_train_steps = num_update_steps_per_epoch * epochs
optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-4)

lr_scheduler = get_scheduler(
    'constant',
    optimizer=optimizer,
    num_warmup_steps=lr_warmup_steps * 1,
    num_training_steps=max_train_steps * 1,
)

In [88]:
for epoch in range(epochs):
    unet.train()
    train_loss = 0.0

    for step, batch in tqdm(enumerate(train_dataloader)):

        model_input = compute_vae_encodings(batch['img_tensors'], vae)
        print('vae computed')
        noise = torch.randn_like(model_input)

        bsz = model_input.shape[0]
        timesteps = torch.randint(
            0, noise_scheduler.config.num_train_timesteps, (bsz,), device=model_input.device
        )
        timesteps = timesteps.long()

        # Add noise to the model input according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        noisy_model_input = noise_scheduler.add_noise(model_input, noise, timesteps)

        def compute_time_ids(original_size, crops_coords_top_left):
            # Adapted from pipeline.StableDiffusionXLPipeline._get_add_time_ids
            target_size = (1024, 1024)
            add_time_ids = list(original_size + crops_coords_top_left + target_size)
            add_time_ids = torch.tensor([add_time_ids])
            add_time_ids = add_time_ids.to(unet.device, dtype=torch.float16)
            return add_time_ids

        add_time_ids = torch.cat(
            [compute_time_ids(s, c) for s, c in zip(batch["original_sizes"], batch["crop_top_lefts"])]
        ).to(unet.device)
        

        unet_added_conditions = {"time_ids": add_time_ids}
        
        prompt_embeds, pooled_prompt_embeds = encode_prompt(text_encoders, tokenizers, batch['prompts'])
        
        unet_added_conditions.update({"text_embeds": pooled_prompt_embeds})

        model_pred = unet(
            noisy_model_input, timesteps.to(unet.device), prompt_embeds, added_cond_kwargs=unet_added_conditions
        ).sample
        print('unet computed')
        # Get the target for loss depending on the prediction type

        if noise_scheduler.config.prediction_type == "epsilon":
            target = noise
        elif noise_scheduler.config.prediction_type == "v_prediction":
            target = noise_scheduler.get_velocity(model_input, noise, timesteps)
        else:
            raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")
        snr_gamma = None

        if snr_gamma is None:
            loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
        else:
            # Compute loss-weights as per Section 3.4 of https://arxiv.org/abs/2303.09556.
            # Since we predict the noise instead of x_0, the original formulation is slightly changed.
            # This is discussed in Section 4.2 of the same paper.
            snr = compute_snr(noise_scheduler, timesteps)
            if noise_scheduler.config.prediction_type == "v_prediction":
                # Velocity objective requires that we add one to SNR values before we divide by them.
                snr = snr + 1
            mse_loss_weights = (
                torch.stack([snr, snr_gamma * torch.ones_like(timesteps)], dim=1).min(dim=1)[0] / snr
            )

            loss = F.mse_loss(model_pred.float(), target.float(), reduction="none")
            loss = loss.mean(dim=list(range(1, len(loss.shape)))) * mse_loss_weights
            loss = loss.mean()

        # Gather the losses across all processes for logging (if we use distributed training).
        # avg_loss = accelerator.gather(loss.repeat(2)).mean()

        train_loss += loss
        print(loss.item())
        # Backpropagate
        loss.backward()
        # if accelerator.sync_gradients:
        #     accelerator.clip_grad_norm_(params_to_optimize, args.max_grad_norm)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()





0it [00:00, ?it/s]

vae computed


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x100096 and 2816x1280)

In [101]:
pooled_prompt_embeds.size()[1] / 2816

35.0

In [96]:
add_time_ids

tensor([[1200.,  961.,  167.,    0., 1024., 1024.]], device='cuda:0',
       dtype=torch.float16)

In [91]:
prompt_embeds.size()

torch.Size([1, 77, 2048])

In [105]:
unet.time_proj.num_channels

320

In [80]:
unet_added_conditions['text_embeds'].size()

torch.Size([2, 98560])

In [ ]:
noisy_model_input, timesteps, prompt_embeds, added_cond_kwargs

In [52]:
prompt_embeds.device

device(type='cpu')

In [ ]:
# Train the model
train(unet, dataloader, epochs=3)  # Adjust epochs as needed

# Step 5: Validation and Inference
# Define a function for validation or inference
def generate_images(model, prompts, num_images=1):
    model.eval()
    generated_images = []
    for prompt in prompts:
        # Your inference logic here
        pass
    return generated_images

# Generate images
sample_prompts = ["A futuristic city", "A landscape painting"]  # Replace with your prompts
generated_images = generate_images(unet, sample_prompts)

# Display or save the generated images
for img in generated_images:
    display(img)  # or save them